In [2]:
!pip install datasets
from datasets import load_dataset


# Chargement du dataset

In [179]:
import pandas as pd
import numpy as np
import nltk
import string
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('wordnet')

from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\thiba\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [180]:
# We split the dataset in two categories train and test
dataset_train = load_dataset('imdb', split='train')
dataset_test = load_dataset('imdb', split='test')


Reusing dataset imdb (C:\Users\thiba\.cache\huggingface\datasets\imdb\plain_text\1.0.0\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)
Reusing dataset imdb (C:\Users\thiba\.cache\huggingface\datasets\imdb\plain_text\1.0.0\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


In [181]:
x_train, y_train, x_test, y_test = dataset_train[:]['text'], dataset_train[:]['label'], dataset_test[:]['text'], dataset_test[:]['label']


Nous modifions maintenant le tableau en dataframe, ce qui permet de plus facilement faire de la visualisation sur le jeu de données

In [182]:
df_train = pd.DataFrame(list(zip(x_train, y_train)), columns=['val', 'label'])
df_test = pd.DataFrame(list(zip(x_test, y_test)), columns=['val', 'label'])
df_train.head(3), df_test.head(3)

(                                                 val  label
 0  Bromwell High is a cartoon comedy. It ran at t...      1
 1  Homelessness (or Houselessness as George Carli...      1
 2  Brilliant over-acting by Lesley Ann Warren. Be...      1,
                                                  val  label
 0  I went and saw this movie last night after bei...      1
 1  Actor turned director Bill Paxton follows up h...      1
 2  As a recreational golfer with some knowledge o...      1)

# Prétraitement

Nous appliquons maintenant un simple stemmer en utilisant NLTK PorterStemmer sur chaqu'un des mots d'une review et ceci sur chaque review

In [183]:
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
x_train_stemmed = df_train['val'].apply(simple_stemmer)
x_test_stemmed = df_test['val'].apply(simple_stemmer)

Nous appliquons pour une nouveau tableau un lemmatizer de NLTK sur chaqu'un des mots d'une review et ceci sur chaque review de film

In [184]:
def lemm(text):
    lemmatizer = WordNetLemmatizer()
    text= ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return text
x_train_lemmed = df_train['val'].apply(lemm)
x_test_lemmed = df_test['val'].apply(lemm)

In [185]:
#exemple entre une review lemmatizer et une non lemmatizer
x_test_lemmed[0]

"I went and saw this movie last night after being coaxed to by a few friend of mine. I'll admit that I wa reluctant to see it because from what I knew of Ashton Kutcher he wa only able to do comedy. I wa wrong. Kutcher played the character of Jake Fischer very well, and Kevin Costner played Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. This one did exactly that. The entire theater (which wa sold out) wa overcome by laughter during the first half of the movie, and were moved to tear during the second half. While exiting the theater I not only saw many woman in tears, but many full grown men a well, trying desperately not to let anyone see them crying. This movie wa great, and I suggest that you go see it before you judge."

In [186]:
x_test[0]

"I went and saw this movie last night after being coaxed to by a few friends of mine. I'll admit that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I was wrong. Kutcher played the character of Jake Fischer very well, and Kevin Costner played Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. This one did exactly that. The entire theater (which was sold out) was overcome by laughter during the first half of the movie, and were moved to tears during the second half. While exiting the theater I not only saw many women in tears, but many full grown men as well, trying desperately not to let anyone see them crying. This movie was great, and I suggest that you go see it before you judge."

Nous modifions maintenant nos tableau de review en TF-IDF(term frequency-inverse document frequency) ceci permet d'avoir une mesure statistique de ce qui est contenue dans une review

In [187]:
#tfidf des tableau sans prétraitement
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
tf_idf_train = tf_idf_vect.fit_transform(x_train)
tf_idf_test = tf_idf_vect.transform(x_test)

In [188]:
#tfidf des tableau stemmmer
tf_idf_vect_stemmed = TfidfVectorizer(ngram_range=(1,2))
tf_idf_train_stemmed = tf_idf_vect_stemmed.fit_transform(x_train_stemmed)
tf_idf_test_stemmed = tf_idf_vect_stemmed.transform(x_test_stemmed)

In [189]:
#tfidf des tableau lemmer
tf_idf_vect_lemmed = TfidfVectorizer(ngram_range=(1,2))
tf_idf_train_lemmed = tf_idf_vect_lemmed.fit_transform(x_train_lemmed)
tf_idf_test_lemmed = tf_idf_vect_lemmed.transform(x_test_lemmed)

# Modele Naive Bayes
Pour le naive bayes nous avons choisit MultinomialNB, car il est adapté au contage de mots.

In [190]:
clf = MultinomialNB(alpha=1)
clf.fit(tf_idf_train,y_train)
y_pred_test = clf.predict(tf_idf_test)
report = classification_report(y_test,y_pred_test)
print(report)

              precision    recall  f1-score   support

           0       0.84      0.91      0.87     12500
           1       0.90      0.83      0.86     12500

    accuracy                           0.87     25000
   macro avg       0.87      0.87      0.87     25000
weighted avg       0.87      0.87      0.87     25000



In [191]:
report = classification_report(y_test,y_pred_test)
print(report)

              precision    recall  f1-score   support

           0       0.84      0.91      0.87     12500
           1       0.90      0.83      0.86     12500

    accuracy                           0.87     25000
   macro avg       0.87      0.87      0.87     25000
weighted avg       0.87      0.87      0.87     25000



In [192]:
clf = MultinomialNB(alpha=1)
clf.fit(tf_idf_train_stemmed,y_train)
y_pred_test_stemmed = clf.predict(tf_idf_test_stemmed)
report_stemmed = classification_report(y_test,y_pred_test_stemmed)
print(report_stemmed)

              precision    recall  f1-score   support

           0       0.84      0.91      0.87     12500
           1       0.90      0.82      0.86     12500

    accuracy                           0.87     25000
   macro avg       0.87      0.87      0.86     25000
weighted avg       0.87      0.87      0.86     25000



In [193]:
clf = MultinomialNB(alpha=1)
clf.fit(tf_idf_train_lemmed,y_train)
y_pred_test_lemmed = clf.predict(tf_idf_test_lemmed)
report_lemmed = classification_report(y_test,y_pred_test_lemmed)
print(report_lemmed)

              precision    recall  f1-score   support

           0       0.84      0.91      0.87     12500
           1       0.90      0.82      0.86     12500

    accuracy                           0.87     25000
   macro avg       0.87      0.87      0.87     25000
weighted avg       0.87      0.87      0.87     25000



#### On peut remarquer que les resultats sont vraiment similaires avec ou sans prétraitement ce qui ne parait pas normal, cependant on obtiens de vraiment bon résultats en général

# Analyse
Maintenant nous allons montrer des résultats dont le Naive bayes s'est trompé, et nous allons essayer de comprendre pourquoi

In [194]:
for i in range(0,2000):
    if y_test[i] != y_pred_test_lemmed[i]:
        if (y_test[i] == 1):
            print("Cela aurait du être positif mais l'algo a eu négatif")
            print(x_test[i])
        break

Cela aurait du être positif mais l'algo a eu négatif
My wife is a mental health therapist and we watched it from beginning to end. I am the typical man and can not stand chick flicks, but this movie is unbelievable. If you want to see what it is like for someone who is going through these type of struggles, this is the movie for you. As I watched it I found myself feeling sorry for him and others like him. <br /><br />***Spoiler*** Plus the fact that all the individuals in the movie including the people in the mental institution were the actual people in real life made it that more real.<br /><br />A must see for someone in the mental health profession!


De cet example dont l'algorithme bayes naive a eu négatif alors que la review était positive, on peut en déduire c'est a cause de la répétition de mot tel que "can not stand" "feeling sorry", la review en général n'est pas négative mais elle ne contient que peu de mot positif, c'est une revue assez neutre c'est pour cela que notre algorithme n'a pas détecter de répétition de mot dit "positif" et en a donc mal conclue que c'était une revue négative

In [195]:
for i in range(12500,25000):
    if y_test[i] != y_pred_test_lemmed[i]:
        if (y_test[i] == 0):
            print("Cela aurait du être négatif mais l'algo a eu positif")
            print(x_test[i])
        break

Cela aurait du être négatif mais l'algo a eu positif
Brass pictures (movies is not a fitting word for them) really are somewhat brassy. Their alluring visual qualities are reminiscent of expensive high class TV commercials. But unfortunately Brass pictures are feature films with the pretense of wanting to entertain viewers for over two hours! In this they fail miserably, their undeniable, but rather soft and flabby than steamy, erotic qualities non withstanding.<br /><br />Senso '45 is a remake of a film by Luchino Visconti with the same title and Alida Valli and Farley Granger in the lead. The original tells a story of senseless love and lust in and around Venice during the Italian wars of independence. Brass moved the action from the 19th into the 20th century, 1945 to be exact, so there are Mussolini murals, men in black shirts, German uniforms or the tattered garb of the partisans. But it is just window dressing, the historic context is completely negligible.<br /><br />Anna Galien

De cet autre example dont l'algorithme bayes naive a eu positif alors que la review était à charactère négatif, même si le commentaire est dans une première partie très négatif avec la présence de mot tel que "fail" "miserably", l'algorithme a due être trompé due a la présence de mot tel que "quality" "love" etc... Mais aussi par tout ce paragraphe où l'auteur décrit avec positivité la qualité de l'actrice, l'algorithme a surement donc vu beaucoup de vocabulaire qu'il a pensé positif comparé à un champ léxical négatif.